# Transformaciones y Acciones en Spark

En Spark, existen dos tipos principales de operaciones:

- **Transformaciones**: crean un nuevo RDD o DataFrame a partir de uno existente. Son *lazy*, es decir, no se ejecutan hasta que se llama una acción. Ejemplos: `map`, `filter`, `select`, `groupBy`.
- **Acciones**: disparan la ejecución de las transformaciones anteriores. Ejemplos: `collect`, `count`, `show`, `save`.


## Dataset de ejemplo

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

# Crear la sesión de Spark
spark = SparkSession.builder.appName("PySpark03").getOrCreate()

In [ ]:
data = [
    ("Alice", "HR", 3000),
    ("Bob", "IT", 4500),
    ("Cathy", "HR", 3200),
    ("David", "IT", 5000),
    ("Eve", "Finance", 4000)
]
columns = ["name", "department", "salary"]

df = spark.createDataFrame(data, columns)
df.show()

## Transformaciones comunes


In [ ]:
# Selección de columnas
df_select = df.select("name", "salary")

# Filtro
df_filtered = df.filter("salary > 3500")

# Agregación
df_grouped = df.groupBy("department").avg("salary")

## Acciones comunes


In [ ]:
# Mostrar resultados
df_filtered.show()

# Contar filas
print("Filas:", df_filtered.count())

# Recoger datos en una lista
print("Collect:", df_filtered.collect())

## Encadenar transformaciones

In [ ]:
df_result = df.filter('salary > 3500').groupBy("department").count()
df_result.show()

## Aggregate

Las agregaciones te permiten resumir o condensar la información de un DataFrame usando funciones como avg, count, max, min, sum, etc.

In [ ]:
df_result = df.groupBy('department').agg(avg('salary').alias('prom_salary'))
df_result.show()

## Ejercicios prácticos

Leemos los datos de _"../data/movies.csv"_ predefiniendo su esquema

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
# Define el schema
schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True),
    StructField("views", IntegerType(), True),
    StructField("quality", StringType(), True)
])

# Leer archivo CSV
df = spark.read.option("header", True).schema(schema).csv("../data/movies.csv")

df.printSchema()
df.show(5)

Aplicamos lo aprendido

In [ ]:
# Conteo total de registros
print("Total de películas:", df.count())

# Filtrar películas con más de 10.000 visualizaciones y seleccionamos solo algunas columnas
df_filtered = df.select("title", "genres", "views", "quality").filter(f.col("views") > 10000)

# Agrupación por promedio de calidad
df_grouped = df_filtered.groupBy("quality").agg(f.avg("views").alias("avg_views"))
df_grouped.orderBy(f.col("avg_views").desc()).show()

In [ ]:
spark.stop()